In [1]:
"""
Advanced Causal Inference with Double Machine Learning (DML)
Uplift / CATE Estimation with Confounding
Author: Sadhana C
"""

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# --------------------------------------------------
# 1. Data Generation (Confounded Setting)
# --------------------------------------------------

np.random.seed(42)

n = 5000 # number of samples
p = 10 # number of features

# Covariates
X = np.random.normal(0, 1, size=(n, p))

# True heterogeneous treatment effect
true_cate = 2 + X[:, 0] - 0.5 * X[:, 1]

# Propensity score (treatment depends on X -> confounding)
logit_p = 0.6 * X[:, 0] - 0.4 * X[:, 2] + 0.3 * X[:, 3]
propensity = 1 / (1 + np.exp(-logit_p))

T = np.random.binomial(1, propensity)

# Outcome
Y = (
    3
    + X @ np.array([1.2, -1.0, 0.8, 0.6, 0.3, 0, 0, 0, 0, 0])
    + true_cate * T
    + np.random.normal(0, 1, n)
)

# Create DataFrame
data = pd.DataFrame(X, columns=[f"X{i}" for i in range(p)])
data["T"] = T
data["Y"] = Y

# --------------------------------------------------
# 2. Double Machine Learning (Cross-Fitting)
# --------------------------------------------------

X_values = data[[f"X{i}" for i in range(p)]].values
Y_values = data["Y"].values
T_values = data["T"].values

kf = KFold(n_splits=5, shuffle=True, random_state=42)

Y_residual = np.zeros(n)
T_residual = np.zeros(n)

# Outcome model: Regularized regression
outcome_model = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", LassoCV(cv=5))
])

# Treatment model: Propensity score
treatment_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    min_samples_leaf=50,
    random_state=42
)

# Cross-fitting
for train_idx, test_idx in kf.split(X_values):
    X_train, X_test = X_values[train_idx], X_values[test_idx]
    Y_train, Y_test = Y_values[train_idx], Y_values[test_idx]
    T_train, T_test = T_values[train_idx], T_values[test_idx]

    # Fit outcome model
    outcome_model.fit(X_train, Y_train)
    Y_hat = outcome_model.predict(X_test)

    # Fit treatment model
    treatment_model.fit(X_train, T_train)
    T_hat = treatment_model.predict_proba(X_test)[:, 1]

    # Residuals (Orthogonalization)
    Y_residual[test_idx] = Y_test - Y_hat
    T_residual[test_idx] = T_test - T_hat

# --------------------------------------------------
# 3. Final CATE Model
# --------------------------------------------------

# Avoid division instability
T_residual = np.where(np.abs(T_residual) < 1e-3, 1e-3, T_residual)

pseudo_outcome = Y_residual / T_residual

cate_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=6,
    min_samples_leaf=40,
    random_state=42
)

cate_model.fit(X_values, pseudo_outcome)
cate_hat = cate_model.predict(X_values)

# --------------------------------------------------
# 4. Baseline Model (Naive Regression)
# --------------------------------------------------

X_baseline = np.column_stack([
    X_values,
    T_values,
    X_values * T_values[:, None]
])

baseline_model = LinearRegression()
baseline_model.fit(X_baseline, Y_values)

baseline_cate_hat = X_values @ baseline_model.coef_[-p:]

# --------------------------------------------------
# 5. Evaluation: PEHE
# --------------------------------------------------

pehe_dml = np.sqrt(np.mean((cate_hat - true_cate) ** 2))
pehe_baseline = np.sqrt(np.mean((baseline_cate_hat - true_cate) ** 2))

# --------------------------------------------------
# 6. Results & Analysis
# --------------------------------------------------

data["Estimated_CATE"] = cate_hat

cate_summary = data["Estimated_CATE"].describe()

top_responders = data.nlargest(5, "Estimated_CATE")[["X0", "X1", "Estimated_CATE"]]
low_responders = data.nsmallest(5, "Estimated_CATE")[["X0", "X1", "Estimated_CATE"]]

# --------------------------------------------------
# 7. Output
# --------------------------------------------------

print("\n========== DML UPLIFT MODEL RESULTS ==========\n")

print("PEHE Comparison")
print("----------------")
print(f"DML PEHE : {pehe_dml:.4f}")
print(f"Baseline PEHE : {pehe_baseline:.4f}")

print("\nCATE Summary Statistics")
print("-----------------------")
print(cate_summary)

print("\nTop Responders (Highest Uplift)")
print("-------------------------------")
print(top_responders)

print("\nLow Responders (Lowest Uplift)")
print("------------------------------")
print(low_responders)

print("\n=============================================\n")



========== DML UPLIFT MODEL RESULTS ==========

PEHE Comparison
----------------
DML PEHE : 0.4657
Baseline PEHE : 2.0004

CATE Summary Statistics
-----------------------
count    5000.000000
mean        1.863343
std         1.310249
min        -3.004369
25%         1.187928
50%         2.008222
75%         2.700034
max         5.193023
Name: Estimated_CATE, dtype: float64

Top Responders (Highest Uplift)
-------------------------------
            X0        X1  Estimated_CATE
402   2.169937 -1.423683        5.193023
3676  2.047309 -2.180441        5.129620
866   2.305947 -1.399342        5.098201
4497  2.461050 -2.196736        5.083067
4255  2.644988 -1.036937        5.082170

Low Responders (Lowest Uplift)
------------------------------
            X0        X1  Estimated_CATE
3439 -2.077645  1.855584       -3.004369
1039 -2.491161  1.216723       -3.001385
3372 -2.356937  0.477716       -2.963069
1530 -2.431121  1.087940       -2.913495
2869 -2.352437  0.238563       -2.886820


